In [3]:
import os
import pandas as pd
import glob
import re
import nltk
import numpy as np

##### Get the Scripts directory

In [4]:
scripts_dir = os.path.join(os.getcwd(),'scripts')

##### Create the structured dataframe for the unstructured texts

In [5]:
column_names = ['Original File','Season','Episode', 'Episode Name','Script']
script_df = pd.DataFrame(columns=column_names)

In [6]:
print("The dataset consists of {} episodes.".format(len(os.listdir(scripts_dir))))

The dataset consists of 228 episodes.


In [7]:
script_df['Original File'] = os.listdir(scripts_dir)

In [8]:
script_df

,Original File,Season,Episode,Episode Name,Script
0,S01E01 Monica Gets A Roommate.txt,NaN,NaN,NaN,NaN
1,S01E02 The Sonogram At The End.txt,NaN,NaN,NaN,NaN
2,S01E03 The Thumb.txt,NaN,NaN,NaN,NaN
3,S01E04 George Stephanopoulos.txt,NaN,NaN,NaN,NaN
4,S01E05 The East German Laundry Detergent.txt,NaN,NaN,NaN,NaN
...,...,...,...,...,...
223,S10E13 Joey Speaks French.txt,NaN,NaN,NaN,NaN
224,S10E14 Princess Consuela.txt,NaN,NaN,NaN,NaN
225,S10E15 Estelle Dies.txt,NaN,NaN,NaN,NaN
226,S10E16 Rachels Going Away Party.txt,NaN,NaN,NaN,NaN


In [38]:
# season_list = []
# episode_list = []
# for index, row in script_df.iterrows():
#     #print(row['Original File'])
#     x = re.search("S[0-9]{2}",row['Original File'])
#     y = re.search("E[0-9]{2}",row['Original File'])
#     y.group
#     try:
#         season_list.append(x.group(0)[1:])
#     except(AttributeError) as e:
#         season_list.append(row['Original File'])  #'s03E25 At The Beach.txt'

##### Get Season Num, Episode Num & Episode Name from filename

In [9]:
season_list = []
episode_list = []
episode_name_list = []
for index, row in script_df.iterrows():
    #print(row['Original File'])
    x = re.findall("S[0-9]{2}",row['Original File'])
    season_list.append(x[0][1:])
    y = re.findall("E[0-9]{2}",row['Original File'])
    episode_list.append(''.join(y))
    z = re.findall("[a-zA-Z\-\.\s]*.txt",row['Original File'])
    episode_name_list.append(z[0][1:-4])

In [10]:
script_df['Season'] = season_list
script_df['Episode'] = episode_list
script_df['Episode Name'] = episode_name_list

In [11]:
script_df

,Original File,Season,Episode,Episode Name,Script
0,S01E01 Monica Gets A Roommate.txt,01,E01,Monica Gets A Roommate,NaN
1,S01E02 The Sonogram At The End.txt,01,E02,The Sonogram At The End,NaN
2,S01E03 The Thumb.txt,01,E03,The Thumb,NaN
3,S01E04 George Stephanopoulos.txt,01,E04,George Stephanopoulos,NaN
4,S01E05 The East German Laundry Detergent.txt,01,E05,The East German Laundry Detergent,NaN
...,...,...,...,...,...
223,S10E13 Joey Speaks French.txt,10,E13,Joey Speaks French,NaN
224,S10E14 Princess Consuela.txt,10,E14,Princess Consuela,NaN
225,S10E15 Estelle Dies.txt,10,E15,Estelle Dies,NaN
226,S10E16 Rachels Going Away Party.txt,10,E16,Rachels Going Away Party,NaN


##### Create the Scripts Column that stores the screenplay text for each episode

In [12]:
text = ""
script_list = []
for file in glob.glob(scripts_dir+"\\*.txt"):
    f = open(file, 'r',encoding='utf8')
    text = f.read()
    script_list.append(text)
    f.close()

In [13]:
script_df['Script'] = script_list

In [14]:
script_df

,Original File,Season,Episode,Episode Name,Script
0,S01E01 Monica Gets A Roommate.txt,01,E01,Monica Gets A Roommate,The One Where Monica Gets a New Roommate (The ...
1,S01E02 The Sonogram At The End.txt,01,E02,The Sonogram At The End,The One With the Sonogram at the End\nWritten ...
2,S01E03 The Thumb.txt,01,E03,The Thumb,The One With the Thumb\nWritten by: Jeffrey As...
3,S01E04 George Stephanopoulos.txt,01,E04,George Stephanopoulos,The One With George Stephanopoulos\nWritten by...
4,S01E05 The East German Laundry Detergent.txt,01,E05,The East German Laundry Detergent,The One With the East German Laundry Detergent...
...,...,...,...,...,...
223,S10E13 Joey Speaks French.txt,10,E13,Joey Speaks French,Friends\n1013 - The One Where Joey Speaks Fren...
224,S10E14 Princess Consuela.txt,10,E14,Princess Consuela,Friends\n1014 - The One With Princess Consuela...
225,S10E15 Estelle Dies.txt,10,E15,Estelle Dies,\n1015 - The One Where Estelle Dies\nTeleplay ...
226,S10E16 Rachels Going Away Party.txt,10,E16,Rachels Going Away Party,Friends\n1016 - The One With Rachel's Going Aw...


##### Dataframe is ready

In [15]:
# Document similarity
# Perform stop word removal
# Stemming

#### It is important to clean the screenplay text for each episode

In [16]:
### Let's look at an example of how a computer sees the text

script_df['Script'][0]

'The One Where Monica Gets a New Roommate (The Pilot-The Uncut Version)\nWritten by: Marta Kauffman & David Crane\n\n\n[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\n\nMonica: There\'s nothing to tell! He\'s just some guy I work with!\n\nJoey: C\'mon, you\'re going out with the guy! There\'s gotta be something wrong with him!\n\nChandler: All right Joey, be nice.  So does he have a hump? A hump and a hairpiece?\n\nPhoebe: Wait, does he eat chalk?\n\n(They all stare, bemused.)\n\nPhoebe: Just, \'cause, I don\'t want her to go through what I went through with Carl- oh!\n\nMonica: Okay, everybody relax. This is not even a date. It\'s just two people going out to dinner and- not having sex.\n\nChandler: Sounds like a date to me.\n\n[Time Lapse]\n\nChandler: Alright, so I\'m back in high school, I\'m standing in the middle of the cafeteria, and I realize I am totally naked.\n\nAll: Oh, yeah. Had that dream.\n\nChandler: Then I look down, and I realize there\'s a phone

In [17]:
#characters of interest, ignore dialogues from other characters
names = ['Phoebe', 'Monica', 'Rachel', 'Chandler', 'Joey', 'Ross']

In [18]:
season01_df = script_df[script_df['Season'] == '01']
season01_df

,Original File,Season,Episode,Episode Name,Script
0,S01E01 Monica Gets A Roommate.txt,01,E01,Monica Gets A Roommate,The One Where Monica Gets a New Roommate (The ...
1,S01E02 The Sonogram At The End.txt,01,E02,The Sonogram At The End,The One With the Sonogram at the End\nWritten ...
2,S01E03 The Thumb.txt,01,E03,The Thumb,The One With the Thumb\nWritten by: Jeffrey As...
3,S01E04 George Stephanopoulos.txt,01,E04,George Stephanopoulos,The One With George Stephanopoulos\nWritten by...
4,S01E05 The East German Laundry Detergent.txt,01,E05,The East German Laundry Detergent,The One With the East German Laundry Detergent...
5,S01E06 The Butt.txt,01,E06,The Butt,The One With the Butt\nWritten by: Adam Chase ...
6,S01E07 The Blackout.txt,01,E07,The Blackout,The One With the Blackout\nWritten by: Jeffrey...
7,S01E08 Nana Dies Twice.txt,01,E08,Nana Dies Twice,The One Where Nana Dies Twice\nWritten by: Mar...
8,S01E09 Underdog Gets Away.txt,01,E09,Underdog Gets Away,The One Where Underdog Gets Away\nWritten by: ...
9,S01E10 The Monkey.txt,01,E10,The Monkey,The One With the Monkey\nWritten by: Adam Chas...


##### Create a DF to store episode name, characters, and their cleaned lines

In [19]:
season_dialogues = pd.DataFrame(columns=["Episode Name","Character", "Line"])

##### Functions to clean text

In [20]:
def remove_scene_details(line):
    clean_line = re.sub("[\(\[].*?[\)\]]", "", line) #regex to remove anything appearing between () and []
    return clean_line

In [21]:
def preprocess_text(data):
    for value in data:
        line = value[1]
        bracket_count = line.count("(")
        #print(bracket_count)
        if bracket_count == 0:
            continue
        else:
            #print(line)
            clean_line = remove_scene_details(line)
            #print(clean_line)
        
        value[1] = clean_line

    return data

In [22]:
def create_episode_dataframe(ep_name, data):
    temp_df = pd.DataFrame(data, columns = ["Character", "Line"])
    temp_df.insert(0, 'Episode Name', ep_name)

    return temp_df

In [23]:
for index, value in season01_df.iterrows():
    ep_name = value['Episode Name']
    #print(ep_name)
    print('Analyzing.....', ep_name)
    lines = value['Script']
    lines_list = lines.splitlines() #Create a list by splitting on '\n'

    lines_list = [x for x in lines_list if x] #removes any '' NULL dialogues

    data = []
    for line in lines_list:
        # print(line)
        if ":" in line:
            splitline = line.split(': ',maxsplit=1) #Take only the first occurence of ':'
            #print(splitline[0])
            if splitline[0] in names:   #if the dialogue is by a character of interest, ignore everything else
                data.append(splitline)   

    #data list is created with 2 values [Character Name, Dialogue]

    #Loop to check if the split on ':' worked or not, ideally nothing should be printed.
    for temp_val in range(len(data)):
        if len(data[temp_val]) > 2:
            print(temp_val)
            print(data[temp_val], '\n')

    data = preprocess_text(data)

    temp_df = create_episode_dataframe(ep_name,data) #this temp_df is created for every episode

    season_dialogues = pd.concat([season_dialogues,temp_df],axis=0) #This is a growing dataframe, new episodes are appended


Analyzing..... Monica Gets A Roommate
Analyzing..... The Sonogram At The End
Analyzing..... The Thumb
Analyzing..... George Stephanopoulos
Analyzing..... The East German Laundry Detergent
Analyzing..... The Butt
Analyzing..... The Blackout
Analyzing..... Nana Dies Twice
Analyzing..... Underdog Gets Away
Analyzing..... The Monkey
Analyzing..... Mrs. Bing
Analyzing..... The Dozen Lasagnas
Analyzing..... The Boobies
Analyzing..... The Candy Hearts
Analyzing..... The Stoned Guy
Analyzing..... The Two Parts Part I
Analyzing..... The Two Parts Part II
Analyzing..... All The Poker
Analyzing..... The Monkey Gets Away
Analyzing..... The Evil Orthodontist
Analyzing..... The Fake Monica
Analyzing..... The Ick Factor
Analyzing..... The Birth
Analyzing..... Rachel Finds Out


In [24]:
season_dialogues.reset_index(drop=True,inplace=True)

In [25]:
season_dialogues

,Episode Name,Character,Line
0,Monica Gets A Roommate,Monica,There's nothing to tell! He's just some guy I ...
1,Monica Gets A Roommate,Joey,"C'mon, you're going out with the guy! There's ..."
2,Monica Gets A Roommate,Chandler,"All right Joey, be nice. So does he have a hu..."
3,Monica Gets A Roommate,Phoebe,"Wait, does he eat chalk?"
4,Monica Gets A Roommate,Phoebe,"Just, 'cause, I don't want her to go through w..."
...,...,...,...
4760,Rachel Finds Out,Ross,"Oh, hey, hey, I got that."
4761,Rachel Finds Out,Ross,No problem. I cannot wait for you to meet my f...
4762,Rachel Finds Out,Ross,Yeah.
4763,Rachel Finds Out,Ross,"No, no, they will. I just... uh..."


##### Sentiment Analysis

In [26]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [27]:
sia = SentimentIntensityAnalyzer()

In [28]:
def calculate_sentiment(dialogue):
    f = sia.polarity_scores(dialogue.strip())
    scores = f['pos'], f['neg'], f['neu']
    return scores

In [29]:
scores_list = []
for index, value in season_dialogues.iterrows():
    #print(value["Line"])
    dialogue = value["Line"]
    scores = calculate_sentiment(dialogue)
    scores_list.append(scores)

In [30]:
scores_df = pd.DataFrame(scores_list, columns=['Positive','Negative','Neutral'])

In [31]:
scores_df

,Positive,Negative,Neutral
0,0.000,0.000,1.000
1,0.000,0.221,0.779
2,0.208,0.000,0.792
3,0.000,0.000,1.000
4,0.000,0.104,0.896
...,...,...,...
4760,0.000,0.000,1.000
4761,0.207,0.327,0.467
4762,1.000,0.000,0.000
4763,0.000,0.524,0.476


##### Merge sentiment scores with the episodes and lines

In [32]:
final_season01df = pd.concat([season_dialogues, scores_df],axis=1) #axis = 1 for column stacking vertically

In [33]:
final_season01df

,Episode Name,Character,Line,Positive,Negative,Neutral
0,Monica Gets A Roommate,Monica,There's nothing to tell! He's just some guy I ...,0.000,0.000,1.000
1,Monica Gets A Roommate,Joey,"C'mon, you're going out with the guy! There's ...",0.000,0.221,0.779
2,Monica Gets A Roommate,Chandler,"All right Joey, be nice. So does he have a hu...",0.208,0.000,0.792
3,Monica Gets A Roommate,Phoebe,"Wait, does he eat chalk?",0.000,0.000,1.000
4,Monica Gets A Roommate,Phoebe,"Just, 'cause, I don't want her to go through w...",0.000,0.104,0.896
...,...,...,...,...,...,...
4760,Rachel Finds Out,Ross,"Oh, hey, hey, I got that.",0.000,0.000,1.000
4761,Rachel Finds Out,Ross,No problem. I cannot wait for you to meet my f...,0.207,0.327,0.467
4762,Rachel Finds Out,Ross,Yeah.,1.000,0.000,0.000
4763,Rachel Finds Out,Ross,"No, no, they will. I just... uh...",0.000,0.524,0.476


In [34]:
final_season01df[final_season01df['Neutral']> 0.5]

,Episode Name,Character,Line,Positive,Negative,Neutral
0,Monica Gets A Roommate,Monica,There's nothing to tell! He's just some guy I ...,0.000,0.000,1.000
1,Monica Gets A Roommate,Joey,"C'mon, you're going out with the guy! There's ...",0.000,0.221,0.779
2,Monica Gets A Roommate,Chandler,"All right Joey, be nice. So does he have a hu...",0.208,0.000,0.792
3,Monica Gets A Roommate,Phoebe,"Wait, does he eat chalk?",0.000,0.000,1.000
4,Monica Gets A Roommate,Phoebe,"Just, 'cause, I don't want her to go through w...",0.000,0.104,0.896
...,...,...,...,...,...,...
4756,Rachel Finds Out,Rachel,Right! You're right!,0.000,0.000,1.000
4757,Rachel Finds Out,Rachel,You know what?,0.000,0.000,1.000
4758,Rachel Finds Out,Rachel,I forgot... I am supposed to pick up a friend ...,0.180,0.133,0.687
4760,Rachel Finds Out,Ross,"Oh, hey, hey, I got that.",0.000,0.000,1.000


In [35]:
final_season01df[final_season01df['Positive'] > 0.5]

,Episode Name,Character,Line,Positive,Negative,Neutral
18,Monica Gets A Roommate,Monica,"Are you okay, sweetie?",0.718,0.0,0.282
24,Monica Gets A Roommate,Ross,Thanks.,1.000,0.0,0.000
27,Monica Gets A Roommate,Phoebe,Fine! Be murky!,0.544,0.0,0.456
28,Monica Gets A Roommate,Ross,"I'll be fine, alright? Really, everyone. I hop...",0.610,0.0,0.390
44,Monica Gets A Roommate,Rachel,"Hi, sure!",0.722,0.0,0.278
...,...,...,...,...,...,...
4719,Rachel Finds Out,Rachel,Yeah.,1.000,0.0,0.000
4747,Rachel Finds Out,Rachel,"Ross, you're like my best friend.",0.767,0.0,0.233
4754,Rachel Finds Out,Ross,"Well, start looking.",0.512,0.0,0.488
4755,Rachel Finds Out,Rachel,Wow.,1.000,0.0,0.000


In [36]:
print(len(final_season01df[final_season01df['Positive']>= 0.5]))
print(len(final_season01df[final_season01df['Negative']>= 0.5]))
print(len(final_season01df[final_season01df['Neutral']>= 0.5]))

439
144
4115


In [37]:
print("Positive Sentiment: ", len(final_season01df[final_season01df['Positive']> 0.5])*100/len(final_season01df))
print("Negative Sentiment: ", len(final_season01df[final_season01df['Negative']> 0.5])*100/len(final_season01df))
print("Neutral Sentiment: ", len(final_season01df[final_season01df['Neutral']>= 0.5])*100/len(final_season01df))

Positive Sentiment:  9.024134312696747
Negative Sentiment:  3.0010493179433366
Neutral Sentiment:  86.3588667366212


##### Positive Sentiment for Characters for Season 01

In [38]:
print("Positive Sentiment % for Ross: ", len(final_season01df[(final_season01df['Positive']> 0.5) & (final_season01df['Character'] == 'Ross')])*100/len(final_season01df[final_season01df['Character']  == 'Ross']))
print("Positive Sentiment % for Rachel: ", len(final_season01df[(final_season01df['Positive']> 0.5) & (final_season01df['Character'] == 'Rachel')])*100/len(final_season01df[final_season01df['Character']  == 'Rachel']))
print("Positive Sentiment % for Joey: ", len(final_season01df[(final_season01df['Positive']> 0.5) & (final_season01df['Character'] == 'Joey')])*100/len(final_season01df[final_season01df['Character']  == 'Joey']))
print("Positive Sentiment % for Chandler: ", len(final_season01df[(final_season01df['Positive']> 0.5) & (final_season01df['Character'] == 'Chandler')])*100/len(final_season01df[final_season01df['Character']  == 'Chandler']))
print("Positive Sentiment % for Phoebe: ", len(final_season01df[(final_season01df['Positive']> 0.5) & (final_season01df['Character'] == 'Phoebe')])*100/len(final_season01df[final_season01df['Character']  == 'Phoebe']))

Positive Sentiment % for Ross:  8.210526315789474
Positive Sentiment % for Rachel:  10.941176470588236
Positive Sentiment % for Joey:  7.5
Positive Sentiment % for Chandler:  7.833537331701346
Positive Sentiment % for Phoebe:  11.477987421383649


##### Negative Sentiment for Characters for Season 01

In [39]:
print("Negative Sentiment % for Ross: ", len(final_season01df[(final_season01df['Negative']> 0.5) & (final_season01df['Character'] == 'Ross')])*100/len(final_season01df[final_season01df['Character']  == 'Ross']))
print("Negative Sentiment % for Rachel: ", len(final_season01df[(final_season01df['Negative']> 0.5) & (final_season01df['Character'] == 'Rachel')])*100/len(final_season01df[final_season01df['Character']  == 'Rachel']))
print("Negative Sentiment % for Joey: ", len(final_season01df[(final_season01df['Negative']> 0.5) & (final_season01df['Character'] == 'Joey')])*100/len(final_season01df[final_season01df['Character']  == 'Joey']))
print("Negative Sentiment % for Chandler: ", len(final_season01df[(final_season01df['Negative']> 0.5) & (final_season01df['Character'] == 'Chandler')])*100/len(final_season01df[final_season01df['Character']  == 'Chandler']))
print("Negative Sentiment % for Phoebe: ", len(final_season01df[(final_season01df['Negative']> 0.5) & (final_season01df['Character'] == 'Phoebe')])*100/len(final_season01df[final_season01df['Character']  == 'Phoebe']))

Negative Sentiment % for Ross:  3.3684210526315788
Negative Sentiment % for Rachel:  2.9411764705882355
Negative Sentiment % for Joey:  2.5
Negative Sentiment % for Chandler:  2.692778457772338
Negative Sentiment % for Phoebe:  2.830188679245283


##### Neutral Sentiment for Characters for Season 01

In [40]:
print("Neutral Sentiment % for Ross: ", len(final_season01df[(final_season01df['Neutral']>= 0.5) & (final_season01df['Character'] == 'Ross')])*100/len(final_season01df[final_season01df['Character']  == 'Ross']))
print("Neutral Sentiment % for Rachel: ", len(final_season01df[(final_season01df['Neutral']>= 0.5) & (final_season01df['Character'] == 'Rachel')])*100/len(final_season01df[final_season01df['Character']  == 'Rachel']))
print("Neutral Sentiment % for Joey: ", len(final_season01df[(final_season01df['Neutral']>= 0.5) & (final_season01df['Character'] == 'Joey')])*100/len(final_season01df[final_season01df['Character']  == 'Joey']))
print("Neutral Sentiment % for Chandler: ", len(final_season01df[(final_season01df['Neutral']>= 0.5) & (final_season01df['Character'] == 'Chandler')])*100/len(final_season01df[final_season01df['Character']  == 'Chandler']))
print("Neutral Sentiment % for Phoebe: ", len(final_season01df[(final_season01df['Neutral']>= 0.5) & (final_season01df['Character'] == 'Phoebe')])*100/len(final_season01df[final_season01df['Character']  == 'Phoebe']))

Neutral Sentiment % for Ross:  86.73684210526316
Neutral Sentiment % for Rachel:  84.3529411764706
Neutral Sentiment % for Joey:  88.75
Neutral Sentiment % for Chandler:  88.37209302325581
Neutral Sentiment % for Phoebe:  83.64779874213836


##### IDEAS

* TF-IDF on Characters' diaglogues for a season
* Consider all the lines by a character for the entire season as a document
* so we have 6 documents in total for 6 characters
* for each character, we get the most important words spoken by them for that season
* can LSA work better than TF-IDF?
* Terms vs concepts
* stop word removal
* TF-IDF and then SVD on it

##### Experimentation Code Below

In [51]:
lines_list
lines_list = [x for x in lines_list if x]

In [52]:
lines_list

['The One Where Monica Gets a New Roommate (The Pilot-The Uncut Version)',
 'Written by: Marta Kauffman & David Crane',
 '[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]',
 "Monica: There's nothing to tell! He's just some guy I work with!",
 "Joey: C'mon, you're going out with the guy! There's gotta be something wrong with him!",
 'Chandler: All right Joey, be nice.  So does he have a hump? A hump and a hairpiece?',
 'Phoebe: Wait, does he eat chalk?',
 '(They all stare, bemused.)',
 "Phoebe: Just, 'cause, I don't want her to go through what I went through with Carl- oh!",
 "Monica: Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.",
 'Chandler: Sounds like a date to me.',
 '[Time Lapse]',
 "Chandler: Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.",
 'All: Oh, yeah. Had that dream.',
 "Chandler: Then I look down, and I realize there's a p

In [57]:
data = []
for line in lines_list:
    # print(line)
    if ":" in line:
        splitline = line.split(': ',maxsplit=1)
        #print(splitline[0])
        if splitline[0] in names:
            data.append(splitline)

In [58]:
data

[['Monica', "There's nothing to tell! He's just some guy I work with!"],
 ['Joey',
  "C'mon, you're going out with the guy! There's gotta be something wrong with him!"],
 ['Chandler',
  'All right Joey, be nice.  So does he have a hump? A hump and a hairpiece?'],
 ['Phoebe', 'Wait, does he eat chalk?'],
 ['Phoebe',
  "Just, 'cause, I don't want her to go through what I went through with Carl- oh!"],
 ['Monica',
  "Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex."],
 ['Chandler', 'Sounds like a date to me.'],
 ['Chandler',
  "Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked."],
 ['Chandler', "Then I look down, and I realize there's a phone... there."],
 ['Joey', 'Instead of...?'],
 ['Chandler', "That's right."],
 ['Joey', 'Never had that dream.'],
 ['Phoebe', 'No.'],
 ['Chandler',
  "All of a sudden, the phone starts to ring. Now I don't know what to do, everybo

In [59]:
for value in range(len(data)):
    if len(data[value]) > 2:
        print(value)
        print(data[value], '\n')

In [60]:
def remove_scene_details(line):
    clean_line = re.sub("[\(\[].*?[\)\]]", "", line)
    return clean_line

In [61]:
for value in data:
    line = value[1]
    bracket_count = line.count("(")
    #print(bracket_count)
    if bracket_count == 0:
        continue
    else:
        print(line)
        clean_line = remove_scene_details(line)
        print(clean_line)
    
    value[1] = clean_line

(mortified) Hi.
 Hi.
(explaining to the others) Carol moved her stuff out today.
 Carol moved her stuff out today.
(to Ross) Let me get you some coffee.
 Let me get you some coffee.
Ooh! Oh! (She starts to pluck at the air just in front of Ross.)
Ooh! Oh! 
Sometimes I wish I was a lesbian... (They all stare at him.) Did I say that out loud?
Sometimes I wish I was a lesbian...  Did I say that out loud?
And I just want a million dollars! (He extends his hand hopefully.)
And I just want a million dollars! 
(pointing at Rachel) De-caff. (to All) Okay, everybody, this is Rachel, another Lincoln High survivor. (to Rachel) This is everybody, this is Chandler, and Phoebe, and Joey, and- you remember my brother Ross?
 De-caff.  Okay, everybody, this is Rachel, another Lincoln High survivor.  This is everybody, this is Chandler, and Phoebe, and Joey, and- you remember my brother Ross?
Oh God... well, it started about a half hour before the wedding. I was in the room where we were keeping all the

In [37]:
x = "(points at the baby she's holding) This is a boy, (points at the baby Chandler is holding) and that's a girl."


In [38]:
re.sub("[\(\[].*?[\)\]]", "", x)

" This is a boy,  and that's a girl."

In [62]:
data

[['Monica', "There's nothing to tell! He's just some guy I work with!"],
 ['Joey',
  "C'mon, you're going out with the guy! There's gotta be something wrong with him!"],
 ['Chandler',
  'All right Joey, be nice.  So does he have a hump? A hump and a hairpiece?'],
 ['Phoebe', 'Wait, does he eat chalk?'],
 ['Phoebe',
  "Just, 'cause, I don't want her to go through what I went through with Carl- oh!"],
 ['Monica',
  "Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex."],
 ['Chandler', 'Sounds like a date to me.'],
 ['Chandler',
  "Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked."],
 ['Chandler', "Then I look down, and I realize there's a phone... there."],
 ['Joey', 'Instead of...?'],
 ['Chandler', "That's right."],
 ['Joey', 'Never had that dream.'],
 ['Phoebe', 'No.'],
 ['Chandler',
  "All of a sudden, the phone starts to ring. Now I don't know what to do, everybo

In [63]:
dialogues_df = pd.DataFrame(data, columns = ["Name", "Line"])

In [66]:
dialogues_df

,Name,Line
0,Monica,There's nothing to tell! He's just some guy I ...
1,Joey,"C'mon, you're going out with the guy! There's ..."
2,Chandler,"All right Joey, be nice. So does he have a hu..."
3,Phoebe,"Wait, does he eat chalk?"
4,Phoebe,"Just, 'cause, I don't want her to go through w..."
...,...,...
260,Chandler,"Did you make it, or are you just serving it?"
261,Rachel,I'm just serving it.
262,Chandler,"Kids, new dream... I'm in Las Vegas."
263,Rachel,"Ugh. Excuse me, could you give this to that g..."


In [79]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


In [81]:
nltk.download([
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",])


[nltk_data] Downloading package names to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\names.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\state_union.zip.
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\twitter_samples.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_

True

In [69]:
lines = dialogues_df["Line"]

In [82]:
sia = SentimentIntensityAnalyzer()

In [92]:
lines[5]

"Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex."

In [116]:
f = sia.polarity_scores("Ugh.  Excuse me, could you give this to that guy over there?  Go ahead.  Thank you.  Sorry.  Okay, Las Vegas.")

In [117]:
f

{'neg': 0.165, 'neu': 0.605, 'pos': 0.23, 'compound': 0.1531}

In [118]:
scores = f['pos'], f['neg'], f['neu']

In [119]:
scores

(0.23, 0.165, 0.605)

In [120]:
def calculate_sentiment(dialogue):
    f = sia.polarity_scores(dialogue.strip())
    scores = f['pos'], f['neg'], f['neu']
    return scores

In [121]:
scores_list = []
for index, value in dialogues_df.iterrows():
    print(value["Line"])
    dialogue = value["Line"]
    scores = calculate_sentiment(dialogue)
    scores_list.append(scores)
    


There's nothing to tell! He's just some guy I work with!
C'mon, you're going out with the guy! There's gotta be something wrong with him!
All right Joey, be nice.  So does he have a hump? A hump and a hairpiece?
Wait, does he eat chalk?
Just, 'cause, I don't want her to go through what I went through with Carl- oh!
Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.
Sounds like a date to me.
Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.
Then I look down, and I realize there's a phone... there.
Instead of...?
That's right.
Never had that dream.
No.
All of a sudden, the phone starts to ring. Now I don't know what to do, everybody starts looking at me.
And they weren't looking at you before?!
Finally, I figure I'd better answer it, and it turns out it's my mother, which is very-very weird, because- she never calls me!
 Hi.
This guy says hello, I wanna kill myself

In [125]:
scores_df = pd.DataFrame(scores_list, columns=['Positive','Negative','Neutral'])

In [126]:
scores_df

,Positive,Negative,Neutral
0,0.000,0.000,1.000
1,0.000,0.221,0.779
2,0.208,0.000,0.792
3,0.000,0.000,1.000
4,0.000,0.104,0.896
...,...,...,...
260,0.000,0.000,1.000
261,0.000,0.000,1.000
262,0.000,0.000,1.000
263,0.230,0.165,0.605


In [128]:
final_df = pd.concat([dialogues_df, scores_df],axis=1)

In [136]:
final_df[final_df['Neutral']> 0.5]

,Name,Line,Positive,Negative,Neutral
0,Monica,There's nothing to tell! He's just some guy I ...,0.000,0.000,1.000
1,Joey,"C'mon, you're going out with the guy! There's ...",0.000,0.221,0.779
2,Chandler,"All right Joey, be nice. So does he have a hu...",0.208,0.000,0.792
3,Phoebe,"Wait, does he eat chalk?",0.000,0.000,1.000
4,Phoebe,"Just, 'cause, I don't want her to go through w...",0.000,0.104,0.896
...,...,...,...,...,...
260,Chandler,"Did you make it, or are you just serving it?",0.000,0.000,1.000
261,Rachel,I'm just serving it.,0.000,0.000,1.000
262,Chandler,"Kids, new dream... I'm in Las Vegas.",0.000,0.000,1.000
263,Rachel,"Ugh. Excuse me, could you give this to that g...",0.230,0.165,0.605
